# Visualizing Wave Reflection

## Reflection of a Triangular Waveform

First, we start off with a simple example of a triangular waveform incident from the left being reflected at the right boundary. Play with the following scenarios by adjusting the `refl_coef` variable:
* $\Gamma = -1$
* $\Gamma = 1$

Observe the behavior of the wave when reflection occurs. 

In [4]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

# set plot params
x_min, x_max = 0, 1
y_min, y_max = -2, 2
x_res = 250     # this many discrete points per unit x

In [5]:
def wave_func(z, t, refl_coef, comp='a'):
    '''
    Return y data points for plotting.
    
    Parameters:
        z: numpy array of z values
        t: discrete time step
        refl_coef: reflection coefficient
        comp: components to return. 'a' for all, 'i' for incident, 
            'r' for reflected, 't' for transmitted.
    '''
    eta = lambda z, t, z0, v: (z-z0) - v*t
    def triag(a, w, z0, v):
        m = a/(w/2)
        tl = m*(eta(z,t,z0,v)) + a
        tr = -m*(eta(z,t,z0,v)) + a
        tl[a*tl<0] = 0
        tr[a*tr<0] = 0
        tl[a*tl>a*a] = 0
        tr[a*tr>=a*a] = 0
        return tl + tr
    
    # plot a Gaussian function with the z and t offsets dealt with by eta.
    a = 1
    w = 0.4
    v = 0.2
    
    waves = {}
    waves['i'] = triag(a, w, 0, v)
    waves['r'] = triag(refl_coef*a, w, x_max*2, -v)
    
    if comp in waves.keys():
        return waves[comp]
    else:
        return waves['i'] + waves['r']

In [6]:
# set the reflection coefficient here
refl_coef = 1.0

def init():
    '''Re-initialize the line plot, expected to be called by FuncAnimation'''
    [l.set_data([], []) for l in lines.values()]
    return (l for l in lines.values())

def animate_line(t):
    '''
    Set the line's x and y plotting points at the specified time step.
    
    Parameters:
        t: discrete time step
    
    Returns:
        Matplotlib line in tuple form.
    '''
    z = np.linspace(x_min, x_max, x_res*x_max)
    [lines[comp].set_data(z,wave_func(z,t,refl_coef,comp)) for comp in ['a','i','r']]
    return (l for l in lines.values())

# set up plotting canvas
lines = {}
fig, ax = plt.subplots(figsize=(10,10),nrows=2)
ax[0].set_title(rf'Superimposed Reflected and Incident Waves ($\Gamma = {refl_coef}$)')
ax[0].set_xlabel(r'$z$ (m)')
ax[0].set_ylabel(r'$V$ (V)')
ax[0].set_xlim((x_min,x_max))
ax[0].set_ylim((y_min,y_max))
lines['a'], = ax[0].plot([], [], lw=2)
ax[1].set_title(rf'Decomposed Waves')
ax[1].set_xlabel(r'$z$ (m)')
ax[1].set_ylabel(r'$V$ (V)')
ax[1].set_xlim((x_min,x_max))
ax[1].set_ylim((y_min,y_max))
lines['i'], = ax[1].plot([], [], c='tab:orange', lw=2, label='Incident')
lines['r'], = ax[1].plot([], [], c='tab:green', lw=2, label='Reflected')
ax[1].legend()
ax[1].grid(True, which='both')
plt.close() # prevent empty plot from showing up

# animation timing settings
anim_interval = 33  # delay between animation frames (ms)
anim_period = 10    # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)
t_vals = np.linspace(0, anim_period, anim_frames)

# generate animation
anim = animation.FuncAnimation(fig, animate_line, init_func=init,
                              frames=t_vals, interval=anim_interval, blit=True)
HTML(anim.to_html5_video())

## Reflection of a Gaussian

In this example, we visualize the reflection of an incident voltage wave packet in the form of a gaussian wave. The transmission line is terminated at the right edge of the plot ($z=1$) by an arbitrary load; the reflection coefficient, $\Gamma$, is set manually by the variable `refl_coef`.

Make the following tweaks and visualize how the reflection behavior changes:
* Run the animation with $\Gamma = -1$, observe that the point at the termination ($z=1$) does not oscillate at all. Consider what kind of load leads to $\Gamma = -1$, and consider the intuition behind why the voltage at that point must stay at 0.
* Run again with any positive $\Gamma$ value (but less than 1), observe that when the voltage wave reaches the termination, the point oscillates beyond the incident voltage amplitude. Is this physically valid?
* Lastly, run again with $\Gamma = 0$. Does the result align with your expectation?

In [7]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

# set plot params
x_min, x_max = 0, 1
y_min, y_max = -2, 2
x_res = 250     # this many discrete points per unit x

In [8]:
def wave_func(z, t, refl_coef, comp='a'):
    '''
    Return y data points for plotting.
    
    Parameters:
        z: numpy array of z values
        t: discrete time step
        refl_coef: reflection coefficient
        comp: components to return. 'a' for all, 'i' for incident, 
            'r' for reflected, 't' for transmitted.
    '''
    eta = lambda z, t, z0, v: (z-z0) - v*t
    gaussian = lambda a, b, c, v: a*np.exp(-(eta(z,t,b,v))**2 / 2*c**2)
    
    # plot a Gaussian function with the z and t offsets dealt with by eta.
    a1, b1, c1, v1 = 1, 0, 12, 0.2
    a2, b2, c2, v2 = refl_coef*a1, 2, c1, -v1
    
    waves = {}
    waves['i'] = gaussian(a1, b1, c1, v1)
    waves['r'] = gaussian(a2, b2, c2, v2)
    
    if comp in waves.keys():
        return waves[comp]
    else:
        return waves['i'] + waves['r']

In [9]:
# set the reflection coefficient here
refl_coef = -1.0

def init():
    '''Re-initialize the line plot, expected to be called by FuncAnimation'''
    [l.set_data([], []) for l in lines.values()]
    return (l for l in lines.values())

def animate_line(t):
    '''
    Set the line's x and y plotting points at the specified time step.
    
    Parameters:
        t: discrete time step
    
    Returns:
        Matplotlib line in tuple form.
    '''
    z = np.linspace(x_min, x_max, x_res*x_max)
    [lines[comp].set_data(z,wave_func(z,t,refl_coef,comp)) for comp in ['a','i','r']]
    return (l for l in lines.values())

# set up plotting canvas
lines = {}
fig, ax = plt.subplots(figsize=(10,10),nrows=2)
ax[0].set_title(rf'Superimposed Reflected and Incident Waves $\Gamma = {refl_coef}$')
ax[0].set_xlabel(r'$z$ (m)')
ax[0].set_ylabel(r'$V$ (V)')
ax[0].set_xlim((x_min,x_max))
ax[0].set_ylim((y_min,y_max))
lines['a'], = ax[0].plot([], [], lw=2)
ax[1].set_title(rf'Decomposed Waves')
ax[1].set_xlabel(r'$z$ (m)')
ax[1].set_ylabel(r'$V$ (V)')
ax[1].set_xlim((x_min,x_max))
ax[1].set_ylim((y_min,y_max))
lines['i'], = ax[1].plot([], [], c='tab:orange', lw=2, label='Incident')
lines['r'], = ax[1].plot([], [], c='tab:green', lw=2, label='Reflected')
ax[1].legend()
ax[1].grid(True, which='both')
plt.close() # prevent empty plot from showing up

# animation timing settings
anim_interval = 33  # delay between animation frames (ms)
anim_period = 12     # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)
t_vals = np.linspace(0, anim_period, anim_frames)

# generate animation
anim = animation.FuncAnimation(fig, animate_line, init_func=init,
                              frames=t_vals, interval=anim_interval, blit=True)
HTML(anim.to_html5_video())

---

# Visualizing Continuous Wave Reflection and Standing Wave

Below, we visualize the reflection of a continuous wave on a transmission line. Once again, the transmission line is terminated at the right edge by an arbitrary load, this time at $z=2$. $\Gamma$ is manually defined by the variable `refl_coef`.

Just like the previous question, run the animation through the following configurations:
* $\Gamma = -1$
* $\Gamma > 0$
* $\Gamma = 0$

When the reflected waves interfere with the incident waves, what do you observe at each $\Gamma$ value?

When $|\Gamma| = 1$, is there still a travelling component?

When $|\Gamma| < 1$, is the wave purely travelling? Or is there still a standing component?

In [16]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
from math import ceil, floor

# set plot params
x_min, x_max = 0, 2
y_min, y_max = -2, 2
x_res = 250     # this many discrete points per unit x

In [17]:
def wave_func(x, t, refl_coef, comp='a'):
    '''
    Return y data points for plotting.
    
    Parameters:
        z: numpy array of z values
        t: discrete time step
        refl_coef: reflection coefficient
        comp: components to return. 'a' for all, 'i' for incident, 
            'r' for reflected, 't' for transmitted.
    '''
        
    eta = lambda x, t, x0, lmbd, v: 2*np.pi/lmbd*(x - x0 - v*t)
    sin_wave = lambda a, x0, lmbd, v: a*np.sin(eta(x,t,x0,lmbd,v))
    sigmoid = lambda p,x0: 1 / (1 + np.exp(p*(x-x0)))
    
    a = 1       # amplitude
    lmbd = 0.3  # lambda, wavelength
    v = 0.3     # velocity
    
    waves = {}
    waves['i'] = sin_wave(a, 0, lmbd, v)
    waves['r'] = sin_wave(-refl_coef*a, 4, lmbd, -v)
    
    waves['i'] = waves['i'] * sigmoid(10, t*v)
    waves['r'] = waves['r'] * sigmoid(-10, 2*x_max-t*v)
    
    if comp in waves.keys():
        return waves[comp]
    else:
        return waves['i'] + waves['r']

In [18]:
# set the reflection coefficient here
refl_coef = 0

def init():
    '''Re-initialize the line plot, expected to be called by FuncAnimation'''
    [l.set_data([], []) for l in lines.values()]
    return (l for l in lines.values())

def animate_line(t):
    '''
    Set the line's x and y plotting points at the specified time step.
    
    Parameters:
        t: discrete time step
    
    Returns:
        Matplotlib line in tuple form.
    '''
    z = np.linspace(x_min, x_max, x_res*x_max)
    [lines[comp].set_data(z,wave_func(z,t,refl_coef,comp)) for comp in ['a','i','r']]
    return (l for l in lines.values())

# set up plotting canvas
lines = {}
fig, ax = plt.subplots(figsize=(10,10),nrows=2)
ax[0].set_title(rf'Wave Reflection with $\Gamma = {refl_coef}$')
ax[0].set_xlabel(r'$z$ (m)')
ax[0].set_ylabel(r'$V$ (V)')
ax[0].set_xlim((x_min,x_max))
ax[0].set_ylim((y_min,y_max))
lines['a'], = ax[0].plot([], [], lw=2)
ax[1].set_title(rf'Decomposed Waves')
ax[1].set_xlabel(r'$z$ (m)')
ax[1].set_ylabel(r'$V$ (V)')
ax[1].set_xlim((x_min,x_max))
ax[1].set_ylim((y_min,y_max))
lines['i'], = ax[1].plot([], [], c='tab:orange', lw=2, label='Incident')
lines['r'], = ax[1].plot([], [], c='tab:green', lw=2, label='Reflected')
ax[1].legend()
ax[1].grid(True, which='both')
plt.close() # prevent empty plot from showing up

# animation timing settings
anim_interval = 33  # delay between animation frames (ms)
anim_period = 15    # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)
t_vals = np.linspace(0, anim_period, anim_frames)

# generate animation
anim = animation.FuncAnimation(fig, animate_line, init_func=init,
                              frames=t_vals, interval=anim_interval, blit=True)
HTML(anim.to_html5_video())

---

# Visualizing Steady State Reflection and Transmission

Lastly, we visualize a system where two transmission lines with arbitrary characteristic impedances are connected with each other. The connection occurs at $z = 2$, which is also marked by a vertical black line. The reflection coefficient, $\Gamma$, is set manually by the variable `refl_coef`.

Observe the system under the following $\Gamma$ settings and consider the reasoning behind the observed behavior.
* At $\Gamma = 0$, verify that all incident wave has been transmitted into line 2. Is there any standing wave component anywhere?
* At $\Gamma = -0.5$, how does the transmitted wave differ from what was observed at $\Gamma = 0$? Observe also the wave pattern in line 1. Is it a standing wave, travelling wave, or both?
* At $\Gamma = 0.5$, observe that the transmitted wave amplitude is greater than that of the incident wave. Does this violate the conservation of energy? Why or why not?
* At $\Gamma = 1$, observe that the animation still shows a transmitted wave at the right side. Why is this physically impossible? 

For all of these $\Gamma$ values, is the waveform continuous at the boundary? Can you explain why it is so?

In [31]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

# set plot params
x_min, x_max = 0, 4
y_min, y_max = -2, 2
x_res = 250     # this many discrete points per unit x

In [32]:
def wave_func(x, t, refl_coef, comp='a'):
    '''
    Return y data points for plotting.
    
    Parameters:
        x: numpy array of x values
        t: discrete time step
        refl_coef: reflection coefficient
        comp: components to return. 'a' for all, 'i' for incident, 
            'r' for reflected, 't' for transmitted.
    '''
    
    eta = lambda x, t, x0, lmbd, v: 2*np.pi/lmbd*(x - x0 - v*t)
    sin_wave = lambda a, x0, lmbd, v: a*np.sin(eta(x,t,x0,lmbd,v))
    
    a = 1       # amplitude
    lmbd = 0.5  # lambda, wavelength
    v = 0.1     # velocity
    
    waves = {}
    waves['i'] = sin_wave(a, 0, lmbd, v) * np.heaviside(-x+x_max/2,0)
    waves['r'] = sin_wave(-refl_coef*a, 2, lmbd, -v) * np.heaviside(-x+x_max/2,0)
    waves['t'] = sin_wave((1+refl_coef)*a, 1, lmbd, v) * np.heaviside(x-x_max/2,1)
    
    if comp in waves.keys():
        w = waves[comp]
        w[w==0] = float('nan')
        return w
    else:
        return waves['i'] + waves['r'] + waves['t']

In [33]:
# set the reflection coefficient here
refl_coef = 1

def init():
    '''Re-initialize the line plot, expected to be called by FuncAnimation'''
    [l.set_data([], []) for l in lines.values()]
    return (l for l in lines.values())

def animate_line(t):
    '''
    Set the line's x and y plotting points at the specified time step.
    
    Parameters:
        t: discrete time step
    
    Returns:
        Matplotlib line in tuple form.
    '''
    x = np.linspace(x_min, x_max, x_res*x_max)
    #y = two_gaussians(x,t,refl_coef)
    y = wave_func(x,t,refl_coef)
    [lines[comp].set_data(x,wave_func(x,t,refl_coef,comp)) for comp in ['a','i','r','t']]
    return (l for l in lines.values())

# set up plotting canvas
lines = {}
fig, ax = plt.subplots(figsize=(10,10),nrows=2)
ax[0].set_title(rf'Steady State Reflection and Transmission with $\Gamma = {refl_coef}$')
ax[0].set_xlabel(r'$z$ (m)')
ax[0].set_ylabel(r'$V$ (V)')
ax[0].set_xlim((x_min,x_max))
ax[0].set_ylim((y_min,y_max))
lines['a'], = ax[0].plot([], [], linewidth=2)
ax[0].axvline(x=2, c='black')
ax[1].set_title(rf'Decomposed Waves')
ax[1].set_xlabel(r'$z$ (m)')
ax[1].set_ylabel(r'$V$ (V)')
ax[1].set_xlim((x_min,x_max))
ax[1].set_ylim((y_min,y_max))
lines['i'], = ax[1].plot([], [], c='tab:orange', lw=2, label='Incident')
lines['r'], = ax[1].plot([], [], c='tab:green', lw=2, label='Reflected')
lines['t'], = ax[1].plot([], [], c='tab:red', lw=2, label='Transmitted')
ax[1].axvline(x=2, c='black')
ax[1].legend()
ax[1].grid(True, which='both')
plt.close() # prevent empty plot from showing up

# animation timing settings
anim_interval = 33  # delay between animation frames (ms)
anim_period = 5     # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)
t_vals = np.linspace(0, anim_period, anim_frames)

# generate animation
anim = animation.FuncAnimation(fig, animate_line, init_func=init,
                              frames=t_vals, interval=anim_interval, blit=True)
HTML(anim.to_html5_video())